# Online Payment Fraud Detection:

In this Notebook, I had created a *Fraud Detection Model* using Random Forest Classifier and also verified whether there is any over-fitting or not. The dataset that i used here is taken from Kaggle and you can download it from here: 
[Click here](https://www.kaggle.com/datasets/jainilcoder/online-payment-fraud-detection/data)

In [1]:
import pandas as pd
df = pd.read_csv("E:/Downloads/Fraud Dataset.csv")
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [2]:
df.shape

(6362620, 11)

In [3]:
df.type.value_counts()

type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64

In [4]:
df.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [5]:
for i in df.columns:
    print(df[i].value_counts())

step
19     51352
18     49579
187    49083
235    47491
307    46968
       ...  
432        4
706        4
693        4
112        2
662        2
Name: count, Length: 743, dtype: int64
type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64
amount
10000000.00    3207
10000.00         88
5000.00          79
15000.00         68
500.00           65
               ... 
8513.48           1
189137.90         1
1615.78           1
128891.19         1
12257.89          1
Name: count, Length: 5316900, dtype: int64
nameOrig
C1902386530    3
C363736674     3
C545315117     3
C724452879     3
C1784010646    3
              ..
C98968405      1
C720209255     1
C1567523029    1
C644777639     1
C1280323807    1
Name: count, Length: 6353307, dtype: int64
oldbalanceOrg
0.00         2102449
184.00           918
133.00           914
195.00           912
164.00           909
              ...   
34577.49           1
184929.21   

In [6]:
df1 = df.drop(["nameOrig","nameDest"],axis = 1)
df1.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [7]:
df2 = df1.drop("isFlaggedFraud",axis = 1)
df2.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0


In [8]:
df2.type.unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [9]:
df2['type'] = df2['type'].map({
    "PAYMENT" : 1,
    "TRANSFER": 2,
    "CASH_OUT": 3,
    "DEBIT" : 4,
    "CASH_IN" : 5
})

In [10]:
df2.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,1,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,1,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,2,181.00,181.0,0.00,0.0,0.0,1
3,1,3,181.00,181.0,0.00,21182.0,0.0,1
4,1,1,11668.14,41554.0,29885.86,0.0,0.0,0


In [30]:
df2[((df2['oldbalanceOrg'] >= df2['newbalanceOrig']) & 
     (df2['newbalanceDest'] >= df2['oldbalanceDest']))].shape

(4918983, 8)

In [31]:
df3 = df2[((df2['oldbalanceOrg'] >= df2['newbalanceOrig']) & 
     (df2['newbalanceDest'] >= df2['oldbalanceDest']))]

df3.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,1,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,1,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,2,181.00,181.0,0.00,0.0,0.0,1
4,1,1,11668.14,41554.0,29885.86,0.0,0.0,0
5,1,1,7817.71,53860.0,46042.29,0.0,0.0,0


In [32]:
df3.shape

(4918983, 8)

In [33]:
df3.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,1,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,1,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,2,181.00,181.0,0.00,0.0,0.0,1
4,1,1,11668.14,41554.0,29885.86,0.0,0.0,0
5,1,1,7817.71,53860.0,46042.29,0.0,0.0,0


In [55]:
df3['relative_diff'] = (df3['oldbalanceOrg'] - df3['newbalanceOrig']) / (df3['oldbalanceOrg'] + 1)
df3.head()

C:\Users\madda\AppData\Local\Temp\ipykernel_30448\3069443271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['relative_diff'] = (df3['oldbalanceOrg'] - df3['newbalanceOrig']) / (df3['oldbalanceOrg'] + 1)


,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,relative_diff
0,1,1,9839.64,170136.0,160296.36,0.0,0.0,0,0.057834
1,1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0.087731
2,1,2,181.00,181.0,0.00,0.0,0.0,1,0.994505
4,1,1,11668.14,41554.0,29885.86,0.0,0.0,0,0.280788
5,1,1,7817.71,53860.0,46042.29,0.0,0.0,0,0.145146


In [56]:
# x = df3.drop("isFraud",axis =1)
x = df3.drop(["isFraud","amount","oldbalanceOrg","newbalanceOrig"],axis =1)
y = df3.isFraud

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

In [59]:
from imblearn.over_sampling import SMOTE
sampler = SMOTE(random_state = 4241)
x_sample,y_sample = sampler.fit_resample(x_scaled,y)

In [60]:
x_train,x_test,y_train,y_test = train_test_split(x_sample,y_sample, test_size = 0.3, random_state = 4241, stratify = y_sample)

In [61]:
x_train.shape,y_train.shape

((6875115, 5), (6875115,))

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Using Logistic Regression:

In [63]:
lg = LogisticRegression()
lg.fit(x_train,y_train)

LogisticRegression()

In [64]:
from sklearn.metrics import classification_report

In [65]:
lg_pred = lg.predict(x_test)
print(classification_report(y_test,lg_pred))

              precision    recall  f1-score   support

           0       0.96      0.70      0.81   1473240
           1       0.76      0.97      0.85   1473239

    accuracy                           0.83   2946479
   macro avg       0.86      0.83      0.83   2946479
weighted avg       0.86      0.83      0.83   2946479



# Using Random Forest Classifier:

In [66]:
rf = RandomForestClassifier(n_estimators = 1)
rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=1)

In [67]:
rf_pred = rf.predict(x_test)
print(classification_report(y_test,rf_pred))

              precision    recall  f1-score   support

           0       0.99      0.93      0.96   1473240
           1       0.94      1.00      0.96   1473239

    accuracy                           0.96   2946479
   macro avg       0.97      0.96      0.96   2946479
weighted avg       0.97      0.96      0.96   2946479



# Checking whether there is any over-fitting or not:

In [68]:
print(rf.score(x_train,y_train))
print(rf.score(x_test,y_test))

0.966950080107751
0.9637920378865759


In [69]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, x_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation scores:", scores)
print("Mean CV accuracy:", scores.mean())


Cross-validation scores: [0.97393716 0.97040122 0.98866055 0.98872382 0.98276683]
Mean CV accuracy: 0.9808979195257098


In [70]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
for train_idx, test_idx in skf.split(x_train, y_train):
    print(f"Train fraud proportion: {y_train.iloc[train_idx].mean()}")
    print(f"Test fraud proportion: {y_train.iloc[test_idx].mean()}")


Train fraud proportion: 0.5000001818151405
Test fraud proportion: 0.4999996363697189
Train fraud proportion: 0.5000001818151405
Test fraud proportion: 0.4999996363697189
Train fraud proportion: 0.5
Test fraud proportion: 0.5000003636302811
Train fraud proportion: 0.5
Test fraud proportion: 0.5000003636302811
Train fraud proportion: 0.5
Test fraud proportion: 0.5000003636302811


In [71]:
from sklearn.metrics import roc_auc_score, f1_score
y_pred = rf.predict(x_test)
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

ROC-AUC Score: 0.9637920485694551
F1 Score: 0.9648969605399274


In [72]:
# Check for potential duplicate rows
print("Duplicate rows after SMOTE:", x.duplicated().sum())

Duplicate rows after SMOTE: 803313


In [73]:
import pandas as pd
feature_importances = pd.Series(rf.feature_importances_, index=x.columns)
print(feature_importances.sort_values(ascending=False))

step              0.281770
type              0.250186
relative_diff     0.236454
oldbalanceDest    0.178718
newbalanceDest    0.052872
dtype: float64


* **Step and Type:**  These features have gained more importance, indicating they play a significant role in identifying fraudulent transactions.
* **Relative Difference:**  The feature you engineered is significant, demonstrating that balance-related information is still crucial but no longer leading to leakage.
* **Old and New Balance Destination:**  These are now less influential, suggesting the model isn't heavily relying on possibly leaky features.

In [74]:
df2['balance_diff'] = df2['oldbalanceOrg'] - df2['amount']
df2['is_fraud_leakage'] = (df2['balance_diff'] == df2['newbalanceOrig']).astype(int)

print(df2['is_fraud_leakage'].mean())  # Proportion of leakage


0.14909314087592848
